# Lab HW 6

In [ ]:
!CT_CUBLAS=1 pip install ctransformers --no-binary ctransformers --q

### **Lab-questions:**

### Note: Run the gpu before using the notebook.

## Q1.Use an open-source LLM (Llama-2) for a question-answering task to perform zero-shot and few-shot prompting(max-3 shots).


### Using the open-sourcce llama-2 for performing question answering task.


In [ ]:
from ctransformers import AutoModelForCausalLM
import time

model_name = "TheBloke/Llama-2-7B-Chat-GGML"; quantized_file = "llama-2-7b-chat.ggmlv3.q4_0.bin"

config = {'max_new_tokens': 256, 'repetition_penalty': 1.1, 'temperature': 0.1, 'stream': True}
llm = AutoModelForCausalLM.from_pretrained(
      model_name,
      model_type="llama",
      model_file=quantized_file,
      **config,
      gpu_layers= 150
)

In [ ]:
zero_shot_prompt = 'Where is Taj Mahal located in the world?'
thr_shot_prompt =     'Where is Taj Mahal located in the world: '    + ' Agra.' +    ' Where is Qutub Minar located in the world: '  + ' Delhi.' + ' Where is Hawa Mahal located in the world: ' + 'Jaipur.' + 'Where is Eiffel tower located in the world: '

tokens0= llm.tokenize(zero_shot_prompt)
st1 = time.time()
NUM_TOKENS1=0
for token in llm.generate(tokens0):
    print(llm.detokenize(token), end='', flush=True)
    NUM_TOKENS1+=1;
    if(NUM_TOKENS1==100):
      break
time_generate = time.time() - st1
print(f"\nTotal Time taken: {time_generate}")

In [ ]:
# three shot prompting
tokens1= llm.tokenize(thr_shot_prompt)
st1 = time.time()
NUM_TOKENS2=0
for token in llm.generate(tokens1):
    print(llm.detokenize(token), end='', flush=True)
    NUM_TOKENS2+=1
    if(NUM_TOKENS2==100):
      break
time_generate = time.time() - st1
print(f"\nTotal Time taken: {time_generate}")

## Q2.Create a RAG setup:
Retrieve relevant paragraphs from the following wikipedia links:
- (https://en.wikipedia.org/wiki/French_First_Republic)
- (https://en.wikipedia.org/wiki/Marie_Antoinette)
- (https://en.wikipedia.org/wiki/French_Revolutionary_Wars)
- (https://en.wikipedia.org/wiki/Palace_of_Versailles) and answer the questions in the given file.

Feed the question and relevant context (paragraphs) to the model.
Generate a detailed answer.

Evaluate on the following:

1.Measure the percentage of answers that exactly match the ground-truth answers.

2.F1-score



In [ ]:
!pip install langchain sentence-transformers faiss-cpu langchain-community wikipedia langchain_chroma chromadb

In [ ]:
import bs4
from langchain_community.document_loaders import WikipediaLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import numpy as np
import json
from langchain.document_loaders import WikipediaLoader
from langchain.embeddings.base import Embeddings
from langchain.vectorstores import Chroma
from ctransformers import AutoModelForCausalLM
from langchain.schema import HumanMessage
import time
import json


docs= []; wiki_pages= ["French First Republic", "Mario Antoinette", "French Revolutionary Wars", "Palace of Versailles"]
for page in wiki_pages:
  loader = WikipediaLoader(query=page, lang="en")
  doc = loader.load()
  docs.append(doc)

splitt = RecursiveCharacterTextSplitter( chunk_size=1000, chunk_overlap=200, add_start_index=True)

all_split= []
for doc in docs:
  cur_split = splitt.split_documents(doc)
  all_split.extend(cur_split)

In [ ]:
with open('data.json', 'r') as file:
    data = json.load(file)

questions= []; answers= []
for item in data:
  questions.append(item['question'])
  answers.append(item['answer'])

In [ ]:
class CTransformersEmbeddings(Embeddings):
    def __init__(self, model):
        self.model = model

    def embed_documents(self, texts):
        """Embeds a list of documents."""
        return [self._embed_text(text) for text in texts]

    def embed_query(self, text):
        """Embeds a single query."""
        return self._embed_text(text)

    def _embed_text(self, text):
        token_ids = self.model.tokenize(text)
        embedding = np.mean(token_ids).astype(np.float32)
        embedding_vector = np.full((768,), embedding)

        return embedding_vector

embedding = CTransformersEmbeddings(model=llm)
vectorstore = Chroma.from_documents(documents=all_split, embedding=embedding)

In [ ]:
def FindAnswer(query):

  docs = vectorstore.similarity_search(query, k=1)
  context= ""
  for doc in docs:
    context+= doc.page_content + " "

  prompt_temp= f"""Take the given Context: {context} and provide the exact answer to the given question:
  Question: {query}
  """
  tokens1= llm.tokenize(prompt_temp)

  c=0; data= ""
  for token in llm.generate(tokens1):
      t1= llm.detokenize(token)
      data+= t1

      c+=1
      if(c==100):
        break
  return data

predictions= []
st1 = time.time()
for query in questions:
  data= FindAnswer(query)
  predictions.append(data)

time_taken = time.time() - st1
print(f"\nTotal Time taken: {time_taken}")

In [ ]:
def exact_match(pred, real):
    return int(pred.strip().lower() == real.strip().lower())

def f1_score_custom(pred, real):
    pred_tok = pred.lower().split(); truth_tok = real.lower().split()
    common = set(pred_tok) & set(truth_tok)
    if not common:
        return 0.0
    precision = len(common) / len(pred_tok)
    recall = len(common) / len(truth_tok)
    return 2 * (precision * recall) / (precision + recall)

exact_matches = 0;  f1_scores = []
for i, pred in enumerate(predictions):
    ground_truth = answers[i]
    exact_matches += exact_match(pred, ground_truth)
    f1 = f1_score_custom(pred, ground_truth)
    f1_scores.append(f1)

num_questions = len(questions)
exact_match_percentage = (exact_matches / num_questions) * 100; average_f1_score = sum(f1_scores) / num_questions

print(f"The Percentage of Exact Match: {exact_match_percentage:.2f}%")
print(f"Finallly, tje average F1 Score has come out to be: {average_f1_score:.2f}")